In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [5]:
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.contrib.layers.python.layers import regularizers
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope

In [6]:
def vgg_arg_scope(weight_decay=0.0005):
    with arg_scope(
        [layers.conv2d,layers_lib.fully_connected],
        activation_fn=nn_ops.relu,
        weights_regularizer=regularizers.l2_regularizer(weight_decay),
        biases_initializer=init_ops.zeros_initializer()):
        with arg_scope([layers.conv2d],padding="SAME") as arg_sc:
            return arg_sc

In [7]:
def vgg_16(inputs,num_classes=1000,is_training=True,dropout_keep_prob=0.5,spatial_squeeze=True,scope="vgg_16"):
    with variable_scope.variable_scope(scope,"vgg_16",[inputs]) as sc:
        end_points_collection = sc.original_name_scope+"_end_points"
        with arg_scope(
            [layers.conv2d,layers_lib.fully_connected,layers_lib.max_pool2d],
            outputs_collection=end_points_collection):
            # 224*224*3
            net = layers_lib.repeat(inputs,2,layers.conv2d,64,[3,3],scope="conv1") #224*224*64
            net = layers_lib.max_pool2d(net,[2,2],scope="pool1") #112*112*64
            
            net = layers_lib.repeat(net,2,layers.conv2d,128,[3,3],scope="conv2") #112*112*128
            net = layers_lib.max_pool2d(net,[2,2],scope="pool2")  #56*56*128
            c
            net = layers_lib.repeat(net,3,layers.conv2d,256,[3,3],scope="conv3") #56*56*256
            net = layers_lib.max_pool2d(net,[2,2],scope="pool3") #28*28*256
            
            net = layers_lib.repeat(net,3,layers.conv2d,512,[3,3],scope="conv4") #28*28*512
            net = layers_lib.max_pool2d(net,[2,2],scope="pool4") #14*14*512
            
            net = layers_lib.repeat(net,3,layers.conv2d,512,[3,3],scope="conv5") #14*14*512
            net = layers_lib.max_pool2d(net,[2,2],scope="pool5") #7*7*512
            
            net = layers.conv2d(net,4096,[7,7],padding='VALID',scope="fc6") #1*4096
            net = layers_lib.dropout(
                net,dropout_keep_prob, is_training=is_training,scope="dropout6")
            
            net = layers_conv2d(net,4096,[1,1],scope="fc7") #1*4096
            net = layers_lib.dropout(
                net,dropout_keep_prob,is_training=is_training,scope="dropout7")
            
            net = layers.conv2d(
                net,num_classes,[1,1],
                activation_fn=None,
                normalizer_fn=None,
                scope="fc8")  #1*num_classes
            end_points = utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                net = array_opt.squeeze(net,[1,2],name="fc8/squeezed")
                end_points[sc.name+"/fc8"] = net
            
            return net,end_points
vgg_16.default_image_size=224        